## Deliverable 2. Create a Customer Travel Destinations Map.

In [37]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mahebourg,MU,-20.4081,57.7000,80.60,78,75,9.22,broken clouds
1,1,Panzhihua,CN,26.5851,101.7128,46.99,65,88,1.43,overcast clouds
2,2,Hudson Bay,CA,52.8500,-102.3842,-22.40,80,43,9.69,scattered clouds
3,3,Kaeo,NZ,-35.1000,173.7833,70.00,72,31,5.99,scattered clouds
4,4,Hobart,AU,-42.8794,147.3294,55.99,58,40,13.80,scattered clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 80


In [10]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
13,13,Santa Rosa,PH,14.3122,121.1114,77.00,78,20,1.14,few clouds
16,16,San Patricio,US,28.0170,-97.5169,75.20,43,40,13.80,scattered clouds
26,26,Labuhan,ID,-6.8844,112.2051,79.02,83,96,12.30,overcast clouds
41,41,Biak,ID,-0.9131,122.8766,77.13,79,100,2.01,overcast clouds
42,42,Isangel,VU,-19.5500,169.2667,78.80,83,20,6.22,few clouds
47,47,Bonthe,SL,7.5264,-12.5050,79.74,82,29,8.12,scattered clouds
48,48,Mananara,MG,-16.1667,49.7667,76.78,88,100,1.83,light rain
50,50,Bengkulu,ID,-3.8004,102.2655,75.51,83,98,6.38,overcast clouds
60,60,Katobu,ID,-4.9408,122.5276,75.40,93,92,1.14,overcast clouds
65,65,Coihaique,CL,-45.5752,-72.0662,78.80,27,0,8.05,clear sky


In [11]:
# 4a. Determine if there are any empty rows.
#preferred_cities_df.isnull()
clean_df.isnull().sum()
# or use preferred_cities_df.notnull().sum()
# else use preferred_cities_df.count()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#If you have some rows with null values, you'll need to drop them using the dropna() method 
#at the end of your filtering statement when you are creating the new DataFrame.
#preferred_cities_df.dropna()

In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
13,Santa Rosa,PH,77.00,few clouds,14.3122,121.1114,
16,San Patricio,US,75.20,scattered clouds,28.0170,-97.5169,
26,Labuhan,ID,79.02,overcast clouds,-6.8844,112.2051,
41,Biak,ID,77.13,overcast clouds,-0.9131,122.8766,
42,Isangel,VU,78.80,few clouds,-19.5500,169.2667,
47,Bonthe,SL,79.74,scattered clouds,7.5264,-12.5050,
48,Mananara,MG,76.78,light rain,-16.1667,49.7667,
50,Bengkulu,ID,75.51,overcast clouds,-3.8004,102.2655,
60,Katobu,ID,75.40,overcast clouds,-4.9408,122.5276,
65,Coihaique,CL,78.80,clear sky,-45.5752,-72.0662,


In [18]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        hotel_df
    except (IndexError):
        print("Hotel not found... skipping.")
        hotels
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [29]:
# 7. Drop the rows where there is no Hotel Name
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [30]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
13,Santa Rosa,PH,77.00,few clouds,14.3122,121.1114,Erlyn’s Sta Rosa City Home
16,San Patricio,US,75.20,scattered clouds,28.0170,-97.5169,"Motel 6 Sinton, TX"
26,Labuhan,ID,79.02,overcast clouds,-6.8844,112.2051,PT PPI Lamongan Tambak C
41,Biak,ID,77.13,overcast clouds,-0.9131,122.8766,Penginapan Kim
42,Isangel,VU,78.80,few clouds,-19.5500,169.2667,Volcano Sea View Tree House
...,...,...,...,...,...,...,...
483,Manokwari,ID,78.58,overcast clouds,-0.8667,134.0833,Swiss Belhotel Manokwari
485,Nanakuli,US,78.80,clear sky,21.3906,-158.1547,Camp Pālehua
498,Alim,PH,78.31,light rain,9.5649,122.4876,Calamai Tropica
501,Mazatlan,MX,75.20,few clouds,23.2167,-106.4167,Gaviana Resort


In [39]:
# 8a. Create the output File (CSV)
# Create the output file (CSV).
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [35]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [36]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
#Description = hotel_df["Current Description"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))